<a href="https://colab.research.google.com/github/joy-hhh/accounting/blob/main/time_monitoring_gdrive.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [113]:
from google.colab import drive
drive.mount("/content/gdrive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [114]:
!ls gdrive/MyDrive/QC/Time_monitoring

20220627_20220930_monitoring.xlsx  score_approve.xlsx
2022_3Q				   time_monitoring_gdrive.ipynb
excess21_result.xlsx		   time_monitoring.ipynb
excess21.xlsx


파일 읽기  
시간 column 속성 부여

In [115]:
import pandas as pd

file_name = "20220627_20220930_monitoring.xlsx"

### 2022.6.27~2022.9.30
df = pd.read_excel("gdrive/MyDrive/QC/Time_monitoring/"+file_name)
df["투입일자"] = pd.to_datetime(df["투입일자"])

df["입력일자/시/분/초"] = df["입력일자/시/분/초"].apply(str)
df["입력일자/시/분/초"] = df["입력일자/시/분/초"].str[:8]
df["입력일자/시/분/초"] = pd.to_datetime(df["입력일자/시/분/초"])

# df.dtypes
df["input"] = df["입력일자/시/분/초"] - df["투입일자"]

입력지연자 : 30일 초과자, 15일 초과자  
30일(1건당 2점), 15일(1건당 1점)
20점 초과 벌점 10점, 10점 초과 20점 이하 벌점 5점

In [116]:
df_input = df[df.구분 == "입력"]
excess30 = df_input[df_input["input"] > "30 days"]
excess30.to_excel("gdrive/MyDrive/QC/Time_monitoring/excess30.xlsx")

excess15 = df_input[(df_input["input"] <= "30 days") & (df_input["input"] > "15 days")]
excess15.to_excel("gdrive/MyDrive/QC/Time_monitoring/excess15.xlsx")

excess30_group = excess30.groupby(['이름'])
excess30_result = pd.DataFrame( {'count' : excess30_group.size()})
excess30_result['score'] = excess30_result['count'] * 2
excess30_result.to_excel("gdrive/MyDrive/QC/Time_monitoring/excess30_result.xlsx")

excess15_group = excess15.groupby(['이름'])
excess15_result = pd.DataFrame( {'count' : excess15_group.size()})
excess15_result['score'] = excess15_result['count']
excess15_result.to_excel("gdrive/MyDrive/QC/Time_monitoring/excess15_result.xlsx")


name_list = df_input['이름'].drop_duplicates().reset_index(drop = True)

score_input = pd.merge(name_list, excess30_result, left_on='이름', right_on='이름', how='left')
score_input = pd.merge(score_input, excess15_result, left_on='이름', right_on='이름', how='left')

score_input.columns = ['이름','30일 초과 횟수','Score_30', '15일 초과 횟수', 'Score_15']
score_input = score_input.fillna(0)
score_input['Score_Total'] = score_input['Score_30'] + score_input['Score_15']
# name_list
score_input = score_input[score_input['Score_Total'] > 0].reset_index(drop = True)

def scoring(s):
    if int(s) > 20:
        return 10
    elif int(s) > 10 and int(s) <=20:
        return 5
    else:
        return 0

score_input['벌점'] = score_input['Score_Total'].map(scoring)
score_input.to_excel("gdrive/MyDrive/QC/Time_monitoring/score_input.xlsx")


승인지연자 : 시간 입력일로부터 21일(14 + 7) 초과자  
40점 초과 벌점 10점, 20점 초과 40점 이하 벌점 5점

In [117]:
df_approve = df[df.구분 != "승인요청"]
df_approved = df_approve.copy()
df_approved["input_shift"] = df_approved["입력일자/시/분/초"].shift()
df_approved["approve_diff"] = df_approved["입력일자/시/분/초"] - df_approved["input_shift"]
df_approved = df_approved[df_approved.구분 == "승인"]

## 99포함하는 코드 제외(행정, 연수, 기타)
df_approved = df_approved[~df_approved['프로젝트코드'].str.contains('99')] 


In [118]:
excess21 = df_approved[df_approved["approve_diff"] > "21 days"]

excess21.to_excel("gdrive/MyDrive/QC/Time_monitoring/excess21.xlsx")

excess21_group = excess21.groupby(['입력(승인)자'])
excess21_result = pd.DataFrame( {'count' : excess21_group.size()})

excess21_result.to_excel("gdrive/MyDrive/QC/Time_monitoring/excess21_result.xlsx")

score_approve = pd.merge(name_list, excess21_result, left_on='이름', right_on='입력(승인)자', how='left')
score_approve = score_approve.dropna().reset_index(drop =True)

def scoring_appr(s):
    if int(s) > 40:
        return 10
    elif int(s) > 20 and int(s) <= 40:
        return 5
    else:
        return 0

score_approve['벌점'] = score_approve['count'].map(scoring_appr)
score_approve.to_excel("gdrive/MyDrive/QC/Time_monitoring/score_approve.xlsx")
